In [2]:
import json
import os
from os.path import join as pjoin
import cv2
import sys

In [3]:
def draw_label(img, bound, color, text=None, put_text=True):
    cv2.rectangle(img, (bound[0], bound[1]), (bound[2], bound[3]), color, 3)
    if put_text:
        # put text with rectangle 
        (w,h),_ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.6, 3)
        cv2.rectangle(img, (bound[0], bound[1] - 40), (bound[0] + w, bound[1] - 40 + h), color, -1)
        cv2.putText(img, text, (bound[0], bound[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3) 


def visualize_list(img, data, show=True):
    board = img.copy()
    is_drawn = False
    
    if data['vertical_groups'] is not None:
        for g in data['vertical_groups']:
            draw_label(board, data['bounds'][g[0]], (0,0,166), 'vertical_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (0,0,166), put_text=False)
                is_drawn = True

    if data['horizontal_groups'] is not None:
        for g in data['horizontal_groups']:
            draw_label(board, data['bounds'][g[0]], (166,0,0), 'horizontal_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,0,0), put_text=False)
                is_drawn = True
                
    if data['multitab_groups'] is not None:
        for g in data['multitab_groups']:
            draw_label(board, data['bounds'][g[0]], (166,0,166), 'multitab_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,0,166), put_text=False)
                is_drawn = True
    
    if data['list_groups'] is not None:
        for g in data['list_groups']:
            draw_label(board, data['bounds'][g[0]], (166,100,255), 'list_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,100,255), put_text=False)
                is_drawn = True        
                
    if data['pageindcator_groups'] is not None:
        for g in data['pageindcator_groups']:
            draw_label(board, data['bounds'][g[0]], (166,166,166), 'pageindcator_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,166,166), put_text=False)
                is_drawn = True     
        
    if show and is_drawn:
        cv2.imshow('data', cv2.resize(board, (500, 800)))
        key = cv2.waitKey(0)
        return key
    return None


def visualize_results(image_name):
    uied_result = cv2.imread(pjoin(result_root, 'uied', image_name + '.jpg'))
    layout_result = cv2.imread(pjoin(result_root, 'layout', image_name + '-list.jpg'))
    cv2.imshow('uied', uied_result)
    cv2.imshow('layout', layout_result)
    

def visualize_ground_truth(image_name, gt_data):
    img_file = pjoin(data_root, image_name + '.jpg')
    img_data = gt_data[image_name]
    img = cv2.imread(img_file)
    
    org_json_file = pjoin(data_root, image_name + '.json')
    org_data = json.load(open(org_json_file))
    org_shape = org_data['activity']['root']['bounds']
    
    img = cv2.resize(img, (1440, 2560))
    return visualize_list(img, img_data)


def cvt_gt_data(gt_data):
    lst = sorted(gt_data['list'].items(), key=lambda x: x[0])
    lst = [l[1] for l in lst]
    return lst

In [4]:
def print_list(gt, result):
    print('*** Ground Truth ***')
    print('Number of list items:', len(gt))
    for item in gt:
        print(item)
    print('*** Result ***')
    print('Number of lists:', len(result))
    for lst in result:
        print('Number of list items:', len(lst))
        for item in lst:
            print(item)

            
def check_correctness(gt, result, is_print=False):
    correct_item_num = 0
    correct_ele_num = 0
    correct_ele_type = 0
    
    for lst in result:
        # level 1: item number in a list
        if len(gt) == len(lst):
            correct_item_num += 1

            # level 2: element number each item
            for i in range(len(gt)):
                if len(gt[i]) == len(lst[i]):
                    correct_ele_num += 1

                    # level 3: element type in each item
                    for j in range(len(gt[i])):
                        if (gt[i][j] == 'TextView' and lst[i][j] == 'Text') or (gt[i][j] != 'TextView' and lst[i][j] != 'Text'):
                            correct_ele_type += 1
            break
            
    if is_print:
        print_list(gt, result)
        print('Correct item number of the list:', correct_item_num)
        print('Correct element number of a item:', correct_ele_num, correct_ele_num / len(gt))
        print('Correct element type number of a item:', correct_ele_type / len(gt))

In [5]:
def is_correct_list(gt, result):
    correct_list_num = 0
    correct_item_num = 0
    correct_ele_type = 0
    
    for rst in result:
        # level 1: item number in a list
        if abs(len(gt) - len(rst)) <= 2:
            # level 2: element number each item
            matched_item_id = 0
            for i in range(len(gt)):
                for j in range(matched_item_id, len(rst)):
                    # compare the element number in the two list items
                    if abs(len(gt[i]) - len(rst[j])) <= 1:
                        correct_item_num += 1
                        matched_item_id = j + 1
            
            if correct_item_num == min(len(gt), len(rst)):
                return True
    return False

In [6]:
data_root = 'E:\\Mulong\\Datasets\\gui\\rico\\combined\\all'
result_root = 'E:\\Mulong\\Result\\rico-layout\\result'
list_data_gt = json.load(open('item_data_class.json'))
gui_gt = json.load(open('valid_group_data.json'))

In [8]:
true_positive = 0
for n, img_name in enumerate(open('select.txt', 'r').readlines()):
    img_name = img_name.replace('\n', '')
    
    list_gt = cvt_gt_data(list_data_gt[img_name])
    list_result = json.load(open(pjoin(result_root, 'layout', img_name + '-list.json')))['list']

    is_correct = is_correct_list(list_gt, list_result)
    if is_correct:
        true_positive += 1
    print(img_name, is_correct, true_positive, '/', n)

2114 True 1 / 0
30783 True 2 / 1
41634 False 2 / 2
13614 False 2 / 3
17314 True 3 / 4
34532 False 3 / 5
30998 False 3 / 6
20553 False 3 / 7
24735 True 4 / 8
14190 False 4 / 9
26644 True 5 / 10
32619 True 6 / 11
41191 False 6 / 12
34917 True 7 / 13
15701 True 8 / 14
24537 False 8 / 15
25173 True 9 / 16
31106 False 9 / 17
9323 True 10 / 18
31099 True 11 / 19
28292 True 12 / 20
7440 True 13 / 21
35228 False 13 / 22
31728 True 14 / 23
35566 True 15 / 24
3183 True 16 / 25
30185 False 16 / 26
21088 False 16 / 27
2060 False 16 / 28
37198 True 17 / 29
12456 False 17 / 30
26903 True 18 / 31
10282 False 18 / 32
27406 False 18 / 33
31517 False 18 / 34
30609 True 19 / 35
24668 False 19 / 36
10283 True 20 / 37
30147 True 21 / 38
7131 True 22 / 39
16631 True 23 / 40
13985 True 24 / 41
8558 False 24 / 42
35573 False 24 / 43
17936 False 24 / 44
39407 True 25 / 45
13563 True 26 / 46
2505 False 26 / 47
4606 False 26 / 48
16341 True 27 / 49
24495 True 28 / 50
31107 True 29 / 51
24250 True 30 / 52
34080 F

8919 True 187 / 408
12802 False 187 / 409
19957 False 187 / 410
39339 True 188 / 411
16655 False 188 / 412
24195 True 189 / 413
23611 False 189 / 414
23177 True 190 / 415
39113 True 191 / 416
16682 False 191 / 417
16697 False 191 / 418
5780 True 192 / 419
21013 False 192 / 420
19215 True 193 / 421
35927 False 193 / 422
10225 True 194 / 423
40374 False 194 / 424
32537 True 195 / 425
26556 False 195 / 426
4688 True 196 / 427
4663 True 197 / 428
15177 False 197 / 429
8850 True 198 / 430
1874 False 198 / 431
37507 True 199 / 432
476 False 199 / 433
27099 False 199 / 434
6672 False 199 / 435
10153 True 200 / 436
30283 False 200 / 437
1135 False 200 / 438
9569 False 200 / 439
12778 True 201 / 440
890 False 201 / 441
22320 True 202 / 442
660 True 203 / 443
27516 True 204 / 444
28625 True 205 / 445
23764 True 206 / 446
6470 True 207 / 447
7020 False 207 / 448
3520 True 208 / 449
38387 False 208 / 450
41120 False 208 / 451
28800 False 208 / 452
4517 False 208 / 453
24790 False 208 / 454
37707 T

42019 True 395 / 799
20844 True 396 / 800
4669 True 397 / 801
8047 True 398 / 802
7374 False 398 / 803
11333 False 398 / 804
29055 True 399 / 805
15974 False 399 / 806
10830 True 400 / 807
16299 False 400 / 808
38170 False 400 / 809
497 False 400 / 810
40008 False 400 / 811
19380 True 401 / 812
38366 True 402 / 813
10367 False 402 / 814
1467 True 403 / 815
17434 False 403 / 816
3502 False 403 / 817
2647 False 403 / 818
39962 True 404 / 819
35180 True 405 / 820
7717 False 405 / 821
24714 False 405 / 822
95 True 406 / 823
29783 True 407 / 824
6151 True 408 / 825
382 False 408 / 826
16229 True 409 / 827
8626 False 409 / 828
36106 True 410 / 829
26315 False 410 / 830
17862 False 410 / 831
36674 False 410 / 832
21334 False 410 / 833
2531 False 410 / 834
18358 False 410 / 835
23480 True 411 / 836
9458 False 411 / 837
23319 False 411 / 838
39427 False 411 / 839
6780 True 412 / 840
20162 False 412 / 841
5289 False 412 / 842
35547 True 413 / 843
11383 False 413 / 844
35235 True 414 / 845
976 Fa

In [9]:
start_point = None
start = False
for img_name in open('select.txt', 'r').readlines()[:200]:
    img_name = img_name.replace('\n', '')
    if start_point is None or img_name == start_point: start = True
    if not start: continue
    print('\n', img_name)
    
    list_gt = cvt_gt_data(list_data_gt[img_name])
    list_result = json.load(open(pjoin(result_root, 'layout', img_name + '-list.json')))['list']

    check_correctness(list_gt, list_result, is_print=True)
    print(is_correct_list(list_gt, list_result))
    
    visualize_results(img_name)
    key = visualize_ground_truth(img_name, gui_gt)
    if key is not None and key != -1 and chr(key) == 'q':
        break
cv2.destroyAllWindows()


 2114
*** Ground Truth ***
Number of list items: 10
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView', 'TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 10
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Text']
['Compo', 'Text']
['Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
Correct item number of the list: 1
Correct element number of a item: 9 0.9
Correct element type number of a item: 0.9
True

 30783
*** Ground Truth ***
Number of list items: 8
['TextView']
['TextView']
['TextView']
['TextView']
['TextView']
['TextView']
['TextView']
['TextView', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 8
['Text']
['Text']
['Text']
['Text']
['Text']
['Text']
['Text']
['Text']
Correct item number of the list: 1
Correct element number of a ite

In [11]:
img_name = '41634'

list_gt = cvt_gt_data(list_data_gt[img_name])
list_result = json.load(open(pjoin(result_root, 'layout', img_name + '-list.json')))['list']

check_correctness(list_gt, list_result, is_print=True)
print(is_correct_list(list_gt, list_result))

visualize_results(img_name)
visualize_ground_truth(img_name, gui_gt)
cv2.destroyAllWindows()

*** Ground Truth ***
Number of list items: 8
['View', 'TextView']
['View', 'TextView']
['View', 'TextView']
['View', 'TextView']
['View', 'TextView']
['View', 'TextView']
['View', 'TextView']
['View', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 4
['Compo', 'Compo', 'Text', 'Text']
['Compo', 'Compo', 'Text', 'Text']
['Compo', 'Compo', 'Text', 'Text']
['Compo', 'Text', 'Text']
Correct item number of the list: 0
Correct element number of a item: 0 0.0
Correct element type number of a item: 0.0
False
